In [ ]:
import numpy as np
from astropy.table import Table, join
from matplotlib import pyplot as plt

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import Span, HoverTool, ColumnDataSource
from bokeh.layouts import gridplot

from parse_sn_data import MASTER_TABLE as master_table

output_notebook()
_basic_tools = "save,pan,box_zoom,reset,wheel_zoom".split(',')

In [ ]:
# Get SMP fit data
smp_fit_params = master_table[
    'CID',
    'x0SALT2zspec', 
    'x0errSALT2zspec', 
    'x1SALT2zspec', 
    'x1errSALT2zspec', 
    'cSALT2zspec', 
    'cerrSALT2zspec']

smp_fit_params.rename_column('CID', 'cid')
smp_fit_params.rename_column('x0SALT2zspec', 'x0_smp')
smp_fit_params.rename_column('x0errSALT2zspec', 'x0_smp_err')
smp_fit_params.rename_column('x1SALT2zspec', 'x1_smp')
smp_fit_params.rename_column('x1errSALT2zspec', 'x1_smp_err')
smp_fit_params.rename_column('cSALT2zspec', 'c_smp')
smp_fit_params.rename_column('cerrSALT2zspec', 'c_smp_err')

# Get SNCosmo fitt data
sncosmo_fits = Table.read('sncosmo_results/summary.csv')

sncosmo_fits.rename_column('x0', 'x0_snc')
sncosmo_fits.rename_column('x1', 'x1_snc')
sncosmo_fits.rename_column('c', 'c_snc')


combined_table = join(sncosmo_fits, smp_fit_params)
combined_table = combined_table[~combined_table['x0_smp'].mask]
for value in ('x0', 'x1', 'c'):
    combined_table['delta_' + value] = \
        (combined_table[value + '_smp'] - combined_table[value + '_snc']) / combined_table[value + '_smp']


# Compare fit parameters between SNCosmo and SDSS

In [ ]:
data_dict = {col.name: np.array(col) for col in combined_table.itercols()}
source = ColumnDataSource(data=data_dict)

figures = []
for value in ('x0', 'x1', 'c'):
    hover = HoverTool(tooltips=[
        ("target", "@cid"),
        ("class", "@class"),
        ("z", "@z"),
        ('z_fit', '@fit_z'),
        ('chi2', '@chi')
    ])

    fig = figure(tools=_basic_tools + [hover, 'box_select', 'lasso_select'], title=value)
    fig.circle(value + '_snc', value + '_smp', source=source, size=4, alpha=.5)
    fig.xaxis.axis_label = value + ' snc'
    fig.yaxis.axis_label = value + ' smp'
    figures.append(fig)

grid = gridplot(figures, ncols=3, plot_width=350, plot_height=350)
show(grid)


In [ ]:
figures = []
for value in ('x0', 'x1', 'c'):
    hover = HoverTool(tooltips=[
        ("target", "@cid"),
        ("class", "@class"),
        ("z", "@z"),
        ('z_fit', '@fit_z'),
        ('chi2', '@chi')
    ])

    fig = figure(tools=_basic_tools + [hover, 'box_select', 'lasso_select'], title=value)
    fig.circle('fit_z', 'delta_' + value, source=source, size=4, alpha=.5)
    fig.xaxis.axis_label = 'Redshift'
    fig.yaxis.axis_label = 'Percent Residual'
    figures.append(fig)

grid = gridplot(figures, ncols=3, plot_width=325, plot_height=325)
show(grid)

